In [2]:
import os
import sys
import json
import time
import datetime
from importlib import reload

import requests

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

import external as ext

In [3]:
desert_main = pd.read_csv('miramar.csv')
baltic_main = pd.read_csv('erangel.csv')
savage_main = pd.read_csv('sanhok.csv')
summerland_main = pd.read_csv('karakin.csv')

In [4]:
desert_main = desert_main.iloc[:,1:]
baltic_main = baltic_main.iloc[:,1:]
savage_main = savage_main.iloc[:,1:]
summerland_main = summerland_main.iloc[:,1:]

In [257]:
def risklevel(x,y,mapname):
    if mapname == 'Desert_Main':
        try:
            risk = desert_main.iloc[y,x]
        except:
            risk = 0
    elif mapname == 'Baltic_Main':
        try:
            risk = baltic_main.iloc[y,x] 
        except:
            risk = 0
    elif mapname == 'Savage_Main':
        try:
            risk = savage_main.iloc[y,x] 
        except:
            risk = 0
    elif mapname == 'Summerland_Main':
        try:
            risk = summerland_main.iloc[y,x]
        except:
            risk = 0
        # correction due to lower max number of players
        risk = risk * 1.5625
    else:
        risk = np.nan
        
    lst.append(risk)

# <font color=red> Preparing DataFrame </font>

In [251]:
import os
import glob

path = 'telemetry_data/2020-04-30/pc'
for filename in glob.glob(os.path.join(path, '*.json')): #only process .JSON files in folder.      
    with open(filename, mode='r') as file:
        json_data = file.read()
        file.close()
    data = []
    head = []
    for i in json.loads(json_data):
        if i["_T"] in ("LogPlayerPosition", "LogParachuteLanding"):
            data.append(i)
        if i["_T"] in ("LogMatchStart"):
            head.append(i)
    

    # get events data into pandas dataframe
    data = pd.read_json(json.dumps(data))
    head = pd.read_json(json.dumps(head))
    
    # get map name
    mn = head.loc[0, 'mapName']
    
    # copy match id to all rows and drop LogMatchDefinition event
    # we will use this later to add use it for future merging tasks
    matchid = json.loads(json_data)[0]["MatchId"]
    data['MatchId'] = matchid

    #reset indexes
    data.reset_index(drop=True, inplace=True)

    # get player information
    from pandas.io.json import json_normalize
    
    # we need to expand the json documents on the character column
    data = pd.concat([data, json_normalize(data['character'])], axis=1).drop(['character', 'zone'], axis=1)
    
    def isGame(x):
        return x['isGame']

    data['isGame'] = data['common'].apply(isGame)
    data = data.drop('common', axis=1)

    # we will use the vehicle to check what players are still in the airplane
    data.vehicle = data.vehicle.apply(lambda x: {} if pd.isna(x) else x)
    data['vehicle'] = json_normalize(data.vehicle)['vehicleType']

    # check how many players' locations we have at each point in time
    # we want to make sure that after this process, all players have harmonized times and locations
    #for i in data.elapsedTime.unique():
        #obs = len(data[data['elapsedTime'] == i])
        #print(f'{i} has {obs} observations')

    # we create a new dataframe with the necessary information to interpolate locations
    data_new_coords = data[['elapsedTime','name','location.x','location.y', 'vehicle']].sort_values(['name','elapsedTime']).drop_duplicates()
    data_new_coords = data_new_coords.reset_index().drop('index', axis=1)
    data_new_coords = data_new_coords.dropna(subset = ['elapsedTime'])
    
    # we round coordinates for ease of calculations, and because a difference in location smaller than a cm is negligible
    data_new_coords['location.x'] = data_new_coords['location.x'].apply(round)
    data_new_coords['location.y'] = data_new_coords['location.y'].apply(round)

    data_new_coords.elapsedTime.unique().astype(int)
    
    # since different elapsed times don't show the location of all players, we want to interpolate the position of the players at each
    # point in time. e.g. if player a is in position (x,y) at second 10.0 and in position (x+10,y+10) at second 20.0
    # we can expect that this player will be at position (x+1,y+1) at second 11.0, and so on
    def getNewCoord(t, n):
        aa = data_new_coords[(data_new_coords['elapsedTime'] < t) & (data_new_coords['name'] == n)] # get previous observations
        bb = data_new_coords[(data_new_coords['elapsedTime'] > t) & (data_new_coords['name'] == n)] # get following observations
        if (bool(aa['location.x'].any())) & (bool(bb['location.x'].any())): # if we have both previous and following observation
            a = aa.iloc[-1] # get last previous observation
            b = bb.iloc[0] # get first following observation
            time = b[0] - a[0] # calculate the time difference
            # get x and y coordinate by calculating movement speed in cm/s and get full coordinate
            new_coord_x = a[2] + ((b[2] - a[2]) / time)*(t-a[0])  
            new_coord_y = a[3] + ((b[3] - a[3]) / time)*(t-a[0])
            # we also get the vehicle to check if a player has jumped from the airplane or not
            in_aircraft = b['vehicle']
            return new_coord_x, new_coord_y, in_aircraft
        elif (not bool(aa['location.x'].any())) & (bool(bb['location.x'].any())):
            # if there is no previous observation, set first following location to be current location
            in_aircraft = b['vehicle']
            return b.iat[0,2], b.iat[0,3], in_aircraft

        # If a player doesn't have a future location, that indicates the player is dead
        elif (bool(aa['location.x'].any())) & (not bool(bb['location.x'].any())):
            # if there is no following observation, we assume the player has died (we hope that only in the match)
            return 'dead', 'dead', 'dead'

    names=[]
    times=[]
    x_coords=[]
    y_coords=[]
    aircraft=[]
    # we will get the location every 10 seconds from second 1 (e.g. 1.0, 11.0, 21.0), and at the last second
    times_list = data_new_coords.elapsedTime.unique().astype(int)
    # we get the times in intervals of 10s from second 1 to the last second of the game
    unique_times_10s = np.arange(1, max(times_list), 10).tolist()
    # we also append the last second, regardless of the 10s interval
    unique_times_10s.append(max(times_list))
    for t in unique_times_10s:
        for n in data_new_coords.name.unique():
            # for every player, we get their location at these times
            a = data_new_coords[(data_new_coords['elapsedTime'] == t) & (data_new_coords['name'] == n)]
            # if the player has a location, we take it
            if a['location.x'].any():
                names.append(n)
                times.append(t)
                x_coords.append(a.iat[0,2])
                y_coords.append(a.iat[0,3])
                aircraft.append(a['vehicle'])
            # if not, we interpolate it
            else:
                x_coord, y_coord, in_aircraft = getNewCoord(t,n)
                names.append(n)
                times.append(t)
                x_coords.append(x_coord)
                y_coords.append(y_coord)
                aircraft.append(in_aircraft)

    # we create the dataframe from these locations
    players_location = pd.DataFrame(list(zip(times,names,x_coords, y_coords, aircraft)),
                 columns=['time','name','location.x','location.y', 'inAircraft'])
    # we also check if the players are or aren't in the aircraft with a binary variable
    players_location['inAircraft'] = players_location['inAircraft'].apply(lambda x: 1 if str(x) == 'TransportAircraft' else 0)


    # we make sure that harmonization of times and observations has been done
    #for i in players_location.time.unique():
        #obs = len(players_location[players_location['time'] == i])
        #print(f'{i} has {obs} observations')

    # encode player names
    # we do this to get the teamIds in the main dataframe
    players = data[['name', 'accountId','teamId']].drop_duplicates()
    players = players.sort_values('teamId').reset_index().drop(['index'], axis=1)


    # we merge these names to get accountId and teamId in the dataframe
    players_location = pd.merge(players_location,
                                players,
                                left_on = 'name',
                                right_on = 'name',
                                how = 'left')
    # moreover, since we are goint to later "asign" a number to the players (e.g. player1, player2), we want to make sure
    # that the criteria is consistent for all teams. Therefore, we do it with the .sort_values method
    # players on each team are ordered with this method inside the team
    players_location = players_location[players_location['time'] >= 0].sort_values(['teamId','time', 'name']).reset_index().drop(['index'], axis=1)

    players_location = players_location.replace('dead',0)

    # convert to 10 meters from cms
    players_location[['location.x','location.y']] = players_location[['location.x','location.y']].apply(lambda x: x.astype(int))
    players_location[['location.x','location.y']] = players_location[['location.x','location.y']].apply(lambda x: x // 1000)

    data = players_location[['location.x','location.y']]

    # convert df to list of tuples
    records = data.to_records(index=False)
    result = list(records)

    cols = ['risk']
    lst = []

    try:
        for i in result:
            risklevel(*i, mapname=mn)

        riskordered = pd.DataFrame(lst, columns=cols)
    
        # export match telemetry data into .csv format
        riskordered.to_csv(path_or_buf=f'{matchid}.csv')
    except:
        continue

KeyboardInterrupt: 

In [273]:
for direct in os.listdir('D:/PUBG/telemetry_data'):
    print(direct)

2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-15
2020-04-16
2020-05-01
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-16


In [275]:
import os
import glob

for direct in os.listdir('../telemetry_data'):
    for d2 in os.listdir(f'../telemetry_data/{direct}'):
        for filename in os.listdir(f'../telemetry_data/{direct}/{d2}'):
        #for filename in glob.glob(os.path.join(path, '*.json')): #only process .JSON files in folder.  
            try:
                with open(f'../telemetry_data/{direct}/{d2}/{filename}', mode='r') as file:
                    json_data = file.read()
                    file.close()
                data = []
                head = []
                for i in json.loads(json_data):
                    if i["_T"] in ("LogPlayerPosition", "LogParachuteLanding"):
                        data.append(i)
                    if i["_T"] in ("LogMatchStart"):
                        head.append(i)


                # get events data into pandas dataframe
                data = pd.read_json(json.dumps(data))
                head = pd.read_json(json.dumps(head))

                # get map name
                mn = head.loc[0, 'mapName']

                # copy match id to all rows and drop LogMatchDefinition event
                # we will use this later to add use it for future merging tasks
                matchid = json.loads(json_data)[0]["MatchId"]
                matchid = matchid.split('.')[-1]
                data['MatchId'] = matchid

                #reset indexes
                data.reset_index(drop=True, inplace=True)

                # get player information
                from pandas.io.json import json_normalize

                # we need to expand the json documents on the character column
                data = pd.concat([data, json_normalize(data['character'])], axis=1).drop(['character', 'zone'], axis=1)

                def isGame(x):
                    return x['isGame']

                data['isGame'] = data['common'].apply(isGame)
                data = data.drop('common', axis=1)

                # we will use the vehicle to check what players are still in the airplane
                data.vehicle = data.vehicle.apply(lambda x: {} if pd.isna(x) else x)
                if 'vehicleType' in data.columns:
                    data['vehicle'] = json_normalize(data.vehicle)['vehicleType']
                else:
                    data['vehicle'] = np.nan

                # check how many players' locations we have at each point in time
                # we want to make sure that after this process, all players have harmonized times and locations
                #for i in data.elapsedTime.unique():
                    #obs = len(data[data['elapsedTime'] == i])
                    #print(f'{i} has {obs} observations')

                # we create a new dataframe with the necessary information to interpolate locations
                data_new_coords = data[['_T','elapsedTime','name','location.x','location.y', 'vehicle']].sort_values(['name','elapsedTime']).drop_duplicates()
                data_new_coords = data_new_coords.reset_index().drop('index', axis=1)
                #data_new_coords = data_new_coords.dropna(subset = ['elapsedTime'])

                # we round coordinates for ease of calculations, and because a difference in location smaller than a cm is negligible
                data_new_coords['location.x'] = data_new_coords['location.x'].apply(round)
                data_new_coords['location.y'] = data_new_coords['location.y'].apply(round)

                event = data_new_coords._T
                names= data_new_coords.name
                times= data_new_coords.elapsedTime
                x_coords = data_new_coords['location.x']
                y_coords= data_new_coords['location.y']
                aircraft= data_new_coords.vehicle

                # we create the dataframe from these locations
                players_location = pd.DataFrame(list(zip(event, times,names,x_coords, y_coords, aircraft)),
                             columns=['event','time','name','location.x','location.y', 'inAircraft'])
                # we also check if the players are or aren't in the aircraft with a binary variable
                players_location['inAircraft'] = players_location['inAircraft'].apply(lambda x: 1 if str(x) == 'TransportAircraft' else 0)


                # we make sure that harmonization of times and observations has been done
                #for i in players_location.time.unique():
                    #obs = len(players_location[players_location['time'] == i])
                    #print(f'{i} has {obs} observations')

                # encode player names
                # we do this to get the teamIds in the main dataframe
                players = data[['name', 'accountId','teamId']].drop_duplicates()
                players = players.sort_values('teamId').reset_index().drop(['index'], axis=1)


                # we merge these names to get accountId and teamId in the dataframe
                players_location = pd.merge(players_location,
                                            players,
                                            left_on = 'name',
                                            right_on = 'name',
                                            how = 'left')
                # moreover, since we are goint to later "asign" a number to the players (e.g. player1, player2), we want to make sure
                # that the criteria is consistent for all teams. Therefore, we do it with the .sort_values method
                # players on each team are ordered with this method inside the team

                players_location = players_location.sort_values(['teamId','time', 'name']).reset_index().drop(['index'], axis=1)

                players_location = players_location.replace('dead',0)

                # convert to 10 meters from cms
                players_location[['location.x','location.y']] = players_location[['location.x','location.y']].apply(lambda x: x.astype(int))
                players_location[['location.x','location.y']] = players_location[['location.x','location.y']].apply(lambda x: x // 1000)

                data = players_location[['location.x','location.y']]

                # convert df to list of tuples
                records = data.to_records(index=False)
                result = list(records)

                cols = ['risk']
                lst = []

                try:
                    for i in result:
                        risklevel(*i, mapname=mn)

                    riskordered = pd.DataFrame(lst, columns=cols)

                    # export match telemetry data into .csv format
                    #riskordered.to_csv(path_or_buf=f'{matchid}.csv')
                    abc = pd.concat([players_location, riskordered], axis=1, sort=False)
                    abc_mean = abc[abc.event != 'LogParachuteLanding'].groupby('name').mean().reset_index()
                    risk_std = abc[abc.event != 'LogParachuteLanding'].groupby('name').std()['risk'].to_list()
                    abc_mean['risk_std'] = risk_std
                    landing_risk = abc[abc.event == 'LogParachuteLanding'][['name','risk']]
                    landing_risk.columns = ['name','landing_risk']
                    abc_final = pd.merge(abc_mean, landing_risk, how='left', on='name')
                    abc_final.insert(0, 'matchId', matchid)
                    abc_final = abc_final.sort_values(['teamId','name'])
                    for team in abc_final.teamId.unique():
                        df = abc_final[abc_final.teamId == team]
                        if len(df) > 4:
                            ghost_players = df[df.time == 0.0]['name'].to_list()
                            df = df[~df.name.isin(ghost_players)]
                            abc_final = abc_final[~abc_final.name.isin(ghost_players)]
                        player_dict = {}
                        for i, player in enumerate(df.name.unique()):
                            player_dict[player] = f'player{i+1}'
                        abc_final = abc_final.replace(player_dict)
                    z = abc_final.pivot_table(values = ['risk','risk_std','landing_risk'], index = ['matchId','teamId'], columns = ['name'])
                    z = z.reset_index()
                    z.columns = z.columns.droplevel(1)
                    if len(z.columns) == 14:
                        z.columns = ['matchId','teamId', 'landing_risk1', 'landing_risk2', 'landing_risk3', 'landing_risk4',
                                    'risk1', 'risk2', 'risk3', 'risk4', 'risk_std1', 'risk_std2', 'risk_std3', 'risk_std4']
                    elif len(z.columns) == 11:
                        z.columns = ['matchId','teamId', 'landing_risk1', 'landing_risk2', 'landing_risk3',
                                    'risk1', 'risk2', 'risk3', 'risk_std1', 'risk_std2', 'risk_std3']
                    elif len(z.columns) == 8:
                        z.columns = ['matchId','teamId', 'landing_risk1', 'landing_risk2',
                                    'risk1', 'risk2', 'risk_std1', 'risk_std2']
                    elif len(z.columns) == 5:
                        z.columns = ['matchId','teamId', 'landing_risk1',
                                    'risk1', 'risk_std1']
                    z.to_csv(f'D:/PUBG/risk_data/{matchid}.csv')
                except (KeyboardInterrupt, SystemExit):
                    raise
            except Exception as e:
                print(f'!!! {filename}: {e}')

!!! 684ca49a-c3e5-49e5-98d4-57d671266288.json: 'character'
!!! b38bd5f5-eca8-4f91-a44c-cfeea9005172.json: 'character'
!!! 2ed63e56-7c7f-4b45-9813-97decefebab0.json: 'character'
!!! da2dbdcb-2c44-4778-b085-149330c26f09.json: 'character'
!!! 2c90b98f-a032-4d93-84b9-4c46fa68f2ef.json: 'character'
!!! 205dc2eb-1368-4b96-8765-95ff72baa676.json: 'character'


NotADirectoryError: [WinError 267] El nombre del directorio no es válido: '../telemetry_data/clean/0005067a-838c-4037-8109-1e3852c23166.csv'

In [277]:
all_filenames = [f'D:/PUBG/risk_data/{i}' for i in os.listdir('D:/PUBG/risk_data')]

In [278]:
#combine all files in the list
risk_csv = pd.concat([pd.read_csv(f, index_col=0) for f in all_filenames ])
#export to csv
risk_csv.to_csv( "D:/PUBG/risk_csv.csv", index=False)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [15]:
risk_csv = pd.read_csv('D:/PUBG/risk_csv.csv')
risk_csv = risk_csv.reset_index(drop=True)

In [16]:
risk_raw = risk_csv.copy()

In [17]:
risk_csv = risk_csv[['matchId','teamId', 'landing_risk1', 'landing_risk2', 'landing_risk3', 'landing_risk4',
                                    'risk1', 'risk2', 'risk3', 'risk4', 'risk_std1', 'risk_std2', 'risk_std3', 'risk_std4']]

In [18]:
risk_csv

,matchId,teamId,landing_risk1,landing_risk2,landing_risk3,landing_risk4,risk1,risk2,risk3,risk4,risk_std1,risk_std2,risk_std3,risk_std4
0,0001759f-70d8-4d4b-8d6d-81def45f2ee3,1.0,35.9375,1.5625,0.0000,12.5,12.687500,27.760417,6.122820,13.728762,23.564241,27.901662,12.887414,24.937355
1,0001759f-70d8-4d4b-8d6d-81def45f2ee3,2.0,150.0000,46.8750,31.2500,37.5,27.812500,24.657012,80.989583,20.632102,22.759540,20.721578,53.007736,16.145332
2,0001759f-70d8-4d4b-8d6d-81def45f2ee3,3.0,200.0000,200.0000,4.6875,NaN,16.535194,13.108553,9.086772,NaN,12.961613,16.731584,9.776795,NaN
3,0001759f-70d8-4d4b-8d6d-81def45f2ee3,4.0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
4,0001759f-70d8-4d4b-8d6d-81def45f2ee3,5.0,21.8750,7.8125,10.9375,0.0,4.468202,5.963010,10.984375,4.890625,7.089531,7.920307,12.189270,6.929788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666000,fffef9cf-5b06-4af6-b8f6-5a5aa80dbd12,200.0,0.0000,0.0000,0.0000,0.0,0.076923,0.021053,0.062893,0.158730,0.268543,0.144321,0.268270,0.409813
666001,fffef9cf-5b06-4af6-b8f6-5a5aa80dbd12,201.0,0.0000,0.0000,0.0000,0.0,0.387755,0.250000,0.663551,3.096154,1.230041,1.609630,1.665091,13.665844
666002,fffef9cf-5b06-4af6-b8f6-5a5aa80dbd12,202.0,0.0000,0.0000,0.0000,0.0,0.905405,0.000000,0.044776,0.000000,0.293648,0.000000,0.270496,0.000000
666003,fffef9cf-5b06-4af6-b8f6-5a5aa80dbd12,203.0,0.0000,0.0000,0.0000,0.0,0.016667,1.391304,0.000000,1.201923,0.129099,1.767496,0.000000,1.685920


In [36]:
a = risk_raw.iloc[:,0:8]
index1 = a.dropna(how='all').index

In [24]:
b = risk_raw.iloc[:,13:21]
b.dropna(subset=['risk.4'])

,risk,risk.1,risk.2,risk.3,risk.4,risk.5,risk.6,risk.7
5770,5.413408,6.067010,4.540541,2.993631,4.718391,NaN,NaN,NaN
13426,10.802083,0.611940,1.153846,1.962963,1.437500,NaN,NaN,NaN
14919,17.931818,1.259259,0.924528,2.307692,2.714286,NaN,NaN,NaN
20607,54.241379,0.233161,17.803922,0.105263,0.200000,NaN,NaN,NaN
24103,0.783333,0.100000,0.725806,0.793651,0.935484,0.454545,0.600000,NaN
...,...,...,...,...,...,...,...,...
657274,14.242424,27.153846,2.066116,39.585714,13.692308,NaN,NaN,NaN
657357,2.052288,26.887640,17.678161,1.483660,30.440476,1.588235,28.954545,1.401316
658127,19.243243,19.609756,12.260000,1.351351,2.475524,4.135417,NaN,NaN
658128,3.842105,1.213415,1.485294,1.054264,1.694805,NaN,NaN,NaN


In [25]:
c = risk_raw.iloc[:,25:33]
c.dropna(subset=['risk_std.4'])

,risk_std,risk_std.1,risk_std.2,risk_std.3,risk_std.4,risk_std.5,risk_std.6,risk_std.7
5770,13.907279,16.777441,9.238379,9.703619,12.955951,NaN,NaN,NaN
13426,18.284901,1.476639,1.603032,4.269077,3.788987,NaN,NaN,NaN
14919,15.761888,2.069294,1.639055,5.140929,5.171619,NaN,NaN,NaN
20607,41.071316,1.119216,23.099457,0.311012,0.987756,NaN,NaN,NaN
24103,1.059128,0.307794,1.161815,1.597970,1.491736,1.184313,1.846761,NaN
...,...,...,...,...,...,...,...,...
657274,26.171347,39.783104,5.317684,29.415313,16.877842,NaN,NaN,NaN
657357,4.101975,26.156607,18.500738,2.690394,27.617295,3.029524,28.219916,3.241412
658127,14.449924,18.873100,13.113025,5.427380,4.320221,5.570340,NaN,NaN
658128,5.816288,2.095175,2.403656,1.737950,2.479532,NaN,NaN,NaN


In [28]:
index = c.dropna(subset=['risk_std.4']).index

In [29]:
index

Int64Index([  5770,  13426,  14919,  20607,  24103,  35521,  38588,  38805,
             46168,  49877,
            ...
            629959, 632072, 632823, 641319, 645336, 657274, 657357, 658127,
            658128, 663233],
           dtype='int64', length=139)

In [43]:
extra_risk = risk_raw.drop(['landing_risk1', 'landing_risk2', 'landing_risk3', 'landing_risk4',
                                    'risk1', 'risk2', 'risk3', 'risk4', 'risk_std1', 'risk_std2', 'risk_std3', 'risk_std4'],axis=1)

In [44]:
extra_risk = extra_risk.iloc[index1,:]
extra_risk = extra_risk.drop(index)
extra_risk = extra_risk[['matchId','teamId', 'landing_risk', 'landing_risk.1', 'landing_risk.2', 'landing_risk.3',
                                    'risk', 'risk.1', 'risk.2', 'risk.3', 'risk_std', 'risk_std.1', 'risk_std.2', 'risk_std.3']]
extra_risk.columns = ['matchId','teamId', 'landing_risk1', 'landing_risk2', 'landing_risk3', 'landing_risk4',
                                    'risk1', 'risk2', 'risk3', 'risk4', 'risk_std1', 'risk_std2', 'risk_std3', 'risk_std4']

In [46]:
risk_csv = pd.concat([risk_csv, extra_risk])

In [48]:
risk_csv.to_csv( "D:/PUBG/risk_csv.csv", index=False)

## Export to .csv

In [268]:
filename

'eb83b1e3-7ca6-46cd-aa45-a45cf6ab9ba0.json'

In [30]:
# export match telemetry data into .csv format
players_location.to_csv(path_or_buf=f'{matchid}.csv')

In [271]:
for direct in os.listdir('D:/PUBG/telemetry_data'):
    for d2 in os.listdir(f'D:/PUBG/telemetry_data/{direct}'):
        for i, filename in enumerate(os.listdir(f'D:/PUBG/telemetry_data/{direct}/{d2}')):
            if filename == 'eb83b1e3-7ca6-46cd-aa45-a45cf6ab9ba0.json':
                print(direct, d2, i)

2020-04-10 console 532
